# Loading the questions

We have different kinds of questions, here im gonna explain about each type.

## Regular Questions:
    Is a question that you have a scale of answers and you could only pick one answer.
    For example:
        Q: How satisfied are you so far?
        A:  1. Not so.
            2. Kind Of.
            3. Very.

## Semi Open Questions:
    Is a question that have multiple answers to select from (you can select only one).
    There is an option to select free text and provide your own answer to the given question.

    for example:
        Q: What is your greatest skill?
        A:  1. Thinking outside the box.
            2. Communication.
            3. Hard working.
            4. Other (explain):

## Multiple Options Questions:
    Similar to a regular question but here you can pick multiple answers.
    For example:
        Q: What are the programming languages that you know (you may pick more than one).
        A:  1. Python.
            2. Java.
            3. C#.
            4. Perl.
            5. Ruby.

## Fully Open Questions:
    In this kind of questions you are given a question without a set of answers to choose from.
    You have to provide your own answer to the question.

    For example:
        Q: Explain why we should hire you.
        A: (can be any answer)

In [1]:
# imports
import pandas as pd
import os
from helper import *

In [2]:
kod_file = pd.read_excel(os.getcwd() + r'\seker\seker_kod.xlsx')
label_file = pd.read_excel(os.getcwd() + r'\seker\seker_label.xlsx')

kod_file.set_index(keys=kod_file.columns[0], drop=True, inplace=True)
label_file.set_index(keys=label_file.columns[0], drop=True, inplace=True)

In [3]:
# create df of shem sheela and mezahe sheela.
q_names = kod_file.columns.tolist()
q_mezahe = kod_file.iloc[0]
questions = list(zip(q_names, q_mezahe))

questions_df = pd.DataFrame(questions, columns=['shem_sheela','mezahe_sheela'])
questions_df = questions_df[questions_df['mezahe_sheela'].isnull() == False]
questions_df.set_index(keys=questions_df.columns[1], drop=False, inplace=True)

In [4]:
kod_df = kod_file[questions_df['shem_sheela']]
kod_df.columns = kod_df.iloc[0].tolist()
kod_df.drop(index=kod_df.index[0], inplace=True)

label_df = label_file[questions_df['shem_sheela']]
label_df.columns = label_df.iloc[0].tolist()
label_df.drop(index=label_df.index[0], inplace=True)

In [5]:
cols = questions_df.index.tolist()
semi_open = []
for i in range(1,len(cols)):
    cols[i] = str(cols[i])
    if cols[i] == cols[i-1]:
        cols[i] = cols[i] + '_dup'
        semi_open.append(cols[i-1])

questions_df.index = cols
for dup in semi_open:
    to_drp = dup + '_dup'
    questions_df.drop(index=to_drp, inplace=True)

In [6]:
kod_df = handle_semi_open(kod_df)
label_df = handle_semi_open(label_df)

data = pd.merge(kod_df, label_df, on=kod_df.index, how='inner', suffixes=('_kod', '_label'))
data.set_index(keys=data.columns[1], drop=False, inplace=True)

In [7]:
frames = []
for q in questions_df.index.to_list():
    df = data[['key_0', str(q) + '_kod', str(q) + '_label']]
    df['mezahe_sheela'] = q
    df.columns = ['mezahe_reshuma', 'kod_tshuva', 'shem_tshuva', 'mezahe_sheela']
    frames.append(df)

tshuvut = pd.concat(frames)
tshuvut.set_index(keys=['mezahe_reshuma', 'mezahe_sheela'], drop=False, inplace=True)

In [8]:
sheelot_ptuchut = tshuvut[tshuvut['kod_tshuva'] == tshuvut['shem_tshuva']]['mezahe_sheela'].unique().tolist()
sheelot_rav_breira = [x for x in tshuvut['mezahe_sheela'].unique().tolist() if len(str(x).split('.')) == 3]

In [37]:
open_q = []
for p in sheelot_ptuchut:
    df = tshuvut.query(f"mezahe_sheela == '{p}'")
    df['kod_tshuva'] = range(len(df['kod_tshuva'].tolist()))
    open_q.append(df)
open_q_df = pd.concat(open_q)

In [39]:
tshuvut.drop(index=open_q_df.index.tolist(),inplace=True)
tshuvut = pd.concat([tshuvut, open_q_df])
tshuvut.sort_index(inplace=True)